In [ ]:
import os
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from python_research.experiments.multiple_feature_learning.builders.keras_builders import build_model, build_settings_for_dataset
from python_research.experiments.multiple_feature_learning.utils.dataset import Dataset
from utils import show_samples_location

DATASET_PATH = "C:\\Users\mmyller.KPLABS\Documents\datasets\pavia\PaviaU_corrected.npy"
DATASET_GT_PATH = "C:\\Users\mmyller.KPLABS\Documents\datasets\pavia\PaviaU_gt.npy"
OUTPUT_PATH = "monte_carlo_3D\\artifact"
TRAIN_SAMPLES_PER_CLASS_COUNT = 250
PIXEL_NEIGHBOURHOOD = (7, 7)
CLASSES_COUNT = 9
PATIENCE = 15
EPOCHS = 200
BATCH_SIZE = 64
os.makedirs("monte_carlo_3D", exist_ok=True)

dataset = Dataset(DATASET_PATH, DATASET_GT_PATH, TRAIN_SAMPLES_PER_CLASS_COUNT, PIXEL_NEIGHBOURHOOD)

bands_count = dataset.x.shape[-1]

settings = build_settings_for_dataset(PIXEL_NEIGHBOURHOOD)

model = build_model(settings, CLASSES_COUNT, bands_count)

print(model.summary())

In [ ]:
# Normalization
dataset.normalize_data()

# Callbacks
early = EarlyStopping(patience=PATIENCE)
checkpoint = ModelCheckpoint(OUTPUT_PATH + "_model", save_best_only=True)

# Model training
model.fit(x=dataset.x_train, y=dataset.y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=False, 
          callbacks=[early, checkpoint], validation_data=[dataset.x_val, dataset.y_val])

best_model = load_model(OUTPUT_PATH + "_model")

# Evaluate test set score
accuracy = best_model.evaluate(x=dataset.x_test, y=dataset.y_test)[1]
print("Test set accuracy: {}".format(accuracy))

In [ ]:
SAMPLES_TO_SHOW_COUNT = 20
show_samples_location(dataset, PIXEL_NEIGHBOURHOOD, SAMPLES_TO_SHOW_COUNT)